This is used as a "third core" to perform seemingly parallel experiments. In this case we do stuff related to experiment 94.

In [1]:
import numpy as np
from numpy import zeros, eye, array, diag, exp, ones, tanh, cosh, dstack, log, hstack, vstack, sinh
from numpy.linalg import solve, norm
from numpy.random import choice, default_rng, randint, randn
from scipy.stats import multivariate_normal as MVN
from scipy.special import logsumexp, ndtr
from scipy.stats import uniform as udist
import math
import time
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib import rc
from ipywidgets.widgets import IntSlider
from ipywidgets.widgets import interact
from warnings import resetwarnings, catch_warnings, filterwarnings
import seaborn as sns
import pickle

from Manifolds.GKManifoldNew import GKManifold, data_generator, find_point_on_manifold, find_point_on_manifold_from_θ
from markov_snippets import MSAdaptive, SMCAdaptive
from tangential_hug_functions import TangentialHugSampler
from tangential_hug_functions import HugTangentialMultivariate
from RWM import RWM
from utils import ESS_univariate
from utils import send_notification_to_iphone

In [2]:
seed = 1234
m = 20     # number of latent variables
d = m + 4  # dimensionality of x-component

In [3]:
def generate_settings(
    d=d, 
    N=2000,
    B=20,
    δ=0.01,
    δmin=1e-3,
    δmax=100.0,
    εmin=1e-16,
    min_pm=1e-4,
    maxiter=500,
    verbose=False,
    εs_fixed=None,
    adaptiveε=True,
    adaptiveδ=False,
    z0_manual=None,
    pm_target=0.23,
    pd_target=0.4,
    dm_target=0.01,
    pm_switch=0.1,
    prior_seed=seed,
    low_memory=True,
    integrator='rwm',
    εprop_switch=0.01,
    ε0_manual=None,
    quantile_value=0.9,
    initialization='manual',
    switch_strategy='pm',
    kernel='uniform',
    prior_scale=7,
    thinning=10,
    burnin=100,
    use_autograd=False,
    stopping_criterion={'εmin', 'pm'},
    resampling_seed=1234,
    mh_kernel_seed=1234,
    resampling_scheme='multinomial',
    prop_hug=0.5,
    projection_method='qr',
    metropolised=False,
    δadaptation_method='ap',
    prop_esjd=0.95,
    adaptiveB=False,
    Bmin=5,
    Bmax=100,
    proxy_ap_metric='dm',
    adaptiveN=False,
    NBbudget=None,
    Badaptation_method=None,
    adaptive_prophug=False,
    min_prop_hug=0.1,
    max_prop_hug=0.9):
    # here d represents m+4
    # Generate the manifold
    θ0        = array([3.0, 1.0, 2.0, 0.5])       # True parameter value on U(0, 10) scale.
    ystar     = data_generator(θ0, m, seed=seed)  # Observed data
    q         = MVN(zeros(d), eye(d))             # Proposal distribution for THUG
    resetwarnings()
    manifold  = GKManifold(ystar, kernel_type=kernel, use_autograd=use_autograd)
    # grab arguments
    arguments = locals()
    return arguments

In [4]:
δmin   = 1e-3
min_pm = 1e-4
pm_switch = 0.1

In [5]:
with open("experiment93/second_particles10k.pkl", "rb") as file:
    P1_10k = pickle.load(file)

### B=20, N=2500, PH=0.3

In [6]:
B20N2k_HAN_03 = MSAdaptive(generate_settings(d=d, N=2500, B=20,
        δ=0.01, δmin=δmin, min_pm=(1/2500), verbose=True, εs_fixed=None, adaptiveε=True, adaptiveδ=False,
        z0_manual=P1_10k[:2500], pm_target=0.23, pm_switch=pm_switch, low_memory=True, integrator='hug_and_nhug',
        εprop_switch=0.01, ε0_manual=200.0, quantile_value=0.5, initialization='manual',
        switch_strategy='pm', kernel='uniform',use_autograd=False, stopping_criterion={'εmin', 'pm', 'maxiter'},
        resampling_scheme='multinomial', maxiter=1000, δadaptation_method=None, adaptiveB=False,
        Bmin=5, Bmax=100, proxy_ap_metric='pm', prop_esjd=0.95, Badaptation_method=None, prop_hug=0.3))
B20N2k_HAN_03_PARTICLES = B20N2k_HAN_03.sample()

Integrator: HUG + NHUG.
Prop Hug  :  0.3
Stopping criterion:  maxiter, εmin, pm.
Resampling: MULTINOMIAL.
Iteration:  1
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 54.7133582196436166
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9480000000000000
	Particle Diversity: 0.4564000000000000
	Moved Diversity: 0.4326672000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  2
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 32.7870673824998917
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9460000000000000
	Particle Diversity: 0.4472000000000000
	Moved Diversity: 0.4230512000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  3
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 25.0695853737435321
	Weights computed and normalized.
	SMC Weights computed and normalized.
	

	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9100000000000000
	Particle Diversity: 0.4588000000000000
	Moved Diversity: 0.4175080000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  24
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 5.9100505528051777
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9028000000000000
	Particle Diversity: 0.4668000000000000
	Moved Diversity: 0.4214270400000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  25
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 5.7698585617515148
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8996000000000000
	Particle Diversity: 0.4508000000000000
	Moved Diversity: 0.4055396800000000
	Step-size kept fixed at: 0.0100000000000000
I

	Trajectories constructed.
	Epsilon: 3.4594557374654493
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8856000000000001
	Particle Diversity: 0.4516000000000000
	Moved Diversity: 0.3999369600000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  47
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 3.4107821451651716
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8912000000000000
	Particle Diversity: 0.4656000000000000
	Moved Diversity: 0.4149427200000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  48
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 3.3780818926800182
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8796000000000000
	Particle Diversity: 0.4568000000000000
	Moved Diversity: 0.4018012

	Trajectories constructed.
	Epsilon: 2.2500987924753972
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9004000000000000
	Particle Diversity: 0.4520000000000000
	Moved Diversity: 0.4069808000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  70
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 2.1776486490613660
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8960000000000000
	Particle Diversity: 0.4520000000000000
	Moved Diversity: 0.4049920000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  71
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 2.1156702421653493
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8895999999999999
	Particle Diversity: 0.4692000000000000
	Moved Diversity: 0.4174003

	Trajectories constructed.
	Epsilon: 1.5510678392998920
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8704000000000000
	Particle Diversity: 0.4692000000000000
	Moved Diversity: 0.4083916800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  93
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 1.5435976226616341
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8800000000000000
	Particle Diversity: 0.4772000000000000
	Moved Diversity: 0.4199360000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  94
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 1.5342097769320846
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8540000000000000
	Particle Diversity: 0.4616000000000000
	Moved Diversity: 0.3942064

...

	Trajectories constructed.
	Epsilon: 0.3052895200398716
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8452000000000000
	Particle Diversity: 0.4720000000000000
	Moved Diversity: 0.3989344000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  898
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.3052605390853140
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8304000000000000
	Particle Diversity: 0.4880000000000000
	Moved Diversity: 0.4052352000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  899
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.3052323888002675
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8472000000000000
	Particle Diversity: 0.4796000000000000
	Moved Diversity: 0.40631

	Trajectories constructed.
	Epsilon: 0.3046064120850700
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8568000000000000
	Particle Diversity: 0.4716000000000000
	Moved Diversity: 0.4040668800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  921
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.3045803665415208
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8296000000000000
	Particle Diversity: 0.4744000000000000
	Moved Diversity: 0.3935622400000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  922
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.3045552215324742
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8320000000000000
	Particle Diversity: 0.4796000000000000
	Moved Diversity: 0.39902

	Trajectories constructed.
	Epsilon: 0.3039493816323187
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8612000000000000
	Particle Diversity: 0.4684000000000000
	Moved Diversity: 0.4033860800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  944
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.3039222458171394
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8480000000000000
	Particle Diversity: 0.4648000000000000
	Moved Diversity: 0.3941504000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  945
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.3038953072715697
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8452000000000000
	Particle Diversity: 0.4704000000000000
	Moved Diversity: 0.39758

	Trajectories constructed.
	Epsilon: 0.3033168001483178
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8448000000000000
	Particle Diversity: 0.4652000000000000
	Moved Diversity: 0.3930009600000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  967
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.3032923676796122
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8440000000000000
	Particle Diversity: 0.4680000000000000
	Moved Diversity: 0.3949920000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  968
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.3032650946306122
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8472000000000000
	Particle Diversity: 0.4712000000000000
	Moved Diversity: 0.39920

	Trajectories constructed.
	Epsilon: 0.3027108525773955
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8376000000000000
	Particle Diversity: 0.4800000000000000
	Moved Diversity: 0.4020480000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  990
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.3026879032383241
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8556000000000000
	Particle Diversity: 0.4708000000000000
	Moved Diversity: 0.4028164800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  991
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.3026594410624278
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8464000000000000
	Particle Diversity: 0.4640000000000000
	Moved Diversity: 0.39272

In [7]:
with open("experiment94/B20N2k_HAN_03_ess.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_03.ESS, file)
    
with open("experiment94/B20N2k_HAN_03_ess_smc.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_03.ESS_SMC, file)
    
with open("experiment94/B20N2k_HAN_03_epsilons.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_03.εs, file)
    
with open("experiment94/B20N2k_HAN_03_pm.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_03.PROP_MOVED, file)
    
with open("experiment94/B20N2k_HAN_03_pd.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_03.P_DIVERSITY, file)
    
with open("experiment94/B20N2k_HAN_03_dm.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_03.DIV_MOVED, file)
    
with open("experiment94/B20N2k_HAN_03_K_RESAMPLED.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_03.K_RESAMPLED, file)
    
with open("experiment94/B20N2k_HAN_03_N_RESAMPLED.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_03.N_RESAMPLED, file)

In [8]:
send_notification_to_iphone("Third core finished: B=20, N=2500, PH=0.3")